In [1]:
import pandas
import spacy
import nltk
from nltk import *
from nltk.tokenize import word_tokenize
from nltk.tag.stanford import StanfordPOSTagger
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
java_path = "C:/Program Files/Java/jdk-18.0.1.1/bin/java.exe"
os.environ["JAVAHOME"] = java_path

jar = "C:/Users/Public/utility/stanford-postagger-full-2020-11-17/stanford-postagger.jar"
model = "C:/Users/Public/utility/stanford-postagger-full-2020-11-17/models/english-bidirectional-distsim.tagger"

In [3]:
pos_tagger = StanfordPOSTagger(model, jar, encoding = "utf-8")

In [4]:
df = pandas.read_csv('C:/Users/sanmeet/Documents/4th_Year/4th_Year_Sem2/BTech_Project/dataset/dataset.csv',encoding="ISO-8859-1",header=[0])
print(df)

                                              sentences  labels
0                     His face became as black as coal.       1
1     My uncle is as blind as a bat without his spec...       1
2                      Naina was as cool as a cucumber.       1
3                   The soldier was as brave as a lion.       1
4                             He is cunning like a fox.       1
...                                                 ...     ...
1810         They like to explore new cities and towns.       0
1811       They like to go on hiking and camping trips.       0
1812  They like to volunteer at local charities and ...       0
1813  They like to watch TV series and binge-watch s...       0
1814  They like to spend time with their families an...       0

[1815 rows x 2 columns]


In [5]:
nlp = spacy.load("en_core_web_sm")

In [6]:
all_stopwords = nlp.Defaults.stop_words
print(all_stopwords)

{'over', 'empty', 'it', 'through', 'thereby', 'nobody', 'seemed', 'give', 'upon', 'whenever', 'being', 'latter', '‘m', 'put', 'could', 'they', 'toward', 'everything', 'you', 'the', 'become', 'will', 'really', 'any', 'alone', 'otherwise', 'per', 'everywhere', 'still', 'too', 'i', 'yourself', 'therein', 'serious', 'than', 'nor', 'anyway', 'often', 'much', 'formerly', 'former', '‘re', 'either', 'hereby', 'full', 'whereafter', '’ll', 'myself', 'besides', 'where', 'might', 'however', 'anything', 'their', 'at', 'twelve', 'moreover', 'himself', 'thereupon', 'though', 'own', 'but', 'becomes', 'forty', 'ever', 'everyone', 'see', 'elsewhere', 'whole', 'two', '’m', '‘s', "n't", 'off', 'becoming', 'made', 'now', 'therefore', 'whereupon', 'may', 'hence', 'so', 'such', 'would', 'quite', 'whence', 'call', 'once', 'me', 'without', 'some', 'n’t', 'also', 'how', 'n‘t', 'anyhow', 'against', 'amount', 'from', 'else', 'during', 'anywhere', 'that', 'is', 'whereby', 'both', 'them', 'were', 'he', 'neither', '

In [7]:
nlp_sentence = nlp(df.loc[0][0])

In [8]:
chunks = list(nlp_sentence.noun_chunks)
print(chunks)

[His face, coal]


In [9]:
nouns = ['NN', 'NNP', 'NNS', 'NNPS']
pronouns = ['PRP', 'PRP$']
adjectives = ['JJ', 'JJR', 'JJS']
verbs = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
helping_verbs = ['be', 'can', 'could', 'dare', 'do', 'have', 'may', 'might', 'must', 'need', 'ought', 'shall', 'should', 'will', 'would']

In [10]:
# get noun chunks
def get_noun_phrase(sentence):
    nlp = spacy.load("en_core_web_sm")
    nlp_sentence = nlp(sentence)
    chunks = list(nlp_sentence.noun_chunks)
    return chunks

In [11]:
target = ""
attribute = ""
source = ""

for i in range(1):
    # for metaphorical sentences
    if df.loc[i]['labels'] == 1:
#         sentence = df.loc[i]['sentences'].lower()
        sentence = "I like listening to music as I do the ironing.".lower()
        
        # for simile sentences having 'as' as a comparator
        if sentence.count(' as ') == 2:
            start = sentence.find(' as ')
            end = sentence.rfind(' as ')
            
            # +4 for skipping ' as '
            attribute = sentence[start+4:end].strip()
            
            target_noun_phrases = get_noun_phrase(sentence[:start])
            source_noun_phrases = get_noun_phrase(sentence[end+4:])
            print(target_noun_phrases)

            if(len(target_noun_phrases) == 0):
                words = nltk.word_tokenize(sentence[:start])
                tagged_words = pos_tagger.tag(words)
                for each in tagged_words:
                    if each[1] in nouns or each[1] in pronouns:
                        target = each[0]
            else:
                target_noun_phrases_lst = word_tokenize(str(target_noun_phrases[0]).strip())
                print("ddd", target_noun_phrases_lst)
                if len(target_noun_phrases_lst) > 1:
                    target_noun_phrases_lst_without_sw = [word for word in target_noun_phrases_lst if not word in all_stopwords]
                    print(target_noun_phrases_lst_without_sw)
                    target = ' '.join(target_noun_phrases_lst_without_sw)
                else:
                    target = target_noun_phrases_lst[0]
            print(source_noun_phrases)
            if(len(source_noun_phrases) == 0):
                words = nltk.word_tokenize(sentence[end+4:])
                tagged_words = pos_tagger.tag(words)
                print(tagged_words)
                for each in tagged_words:
                    if each[1] in nouns or each[1] in pronouns:
                        source = each[0]
            else:
                source_noun_phrases_lst = word_tokenize(str(source_noun_phrases[0]).strip())
                print(source_noun_phrases_lst)
                if len(source_noun_phrases_lst) > 1:
                    source_noun_phrases_lst_without_sw = [word for word in source_noun_phrases_lst if not word in all_stopwords]
                    source = ' '.join(source_noun_phrases_lst_without_sw)
                else:
                    source = source_noun_phrases_lst[0]
                
        # for simile sentences having 'like' as a comparator
        if sentence.count(' like ') == 1:
            like_index = sentence.find(' like ')
            before_like = sentence[:like_index]
            after_like = sentence[like_index+6:]

            words = nltk.word_tokenize(before_like)
            tagged_words = pos_tagger.tag(words)
            
            print(tagged_words)

            for each in tagged_words:
                if each[1] in adjectives:
                    attribute = each[0]
                elif each[1] in verbs:
                    present_form = WordNetLemmatizer().lemmatize(each[0],'v')
                    if present_form not in helping_verbs:
                        attribute = present_form

            attribute_index = before_like.find(attribute)
            target_noun_phrases = get_noun_phrase(before_like[:attribute_index])
            source_noun_phrases = get_noun_phrase(after_like)

            if(len(target_noun_phrases) == 0):
                words = nltk.word_tokenize(before_like[:attribute_index])
                tagged_words = pos_tagger.tag(words)
                for each in tagged_words:
                    if each[1] in nouns or each[1] in pronouns:
                        target = each[0]
            else:
                target_noun_phrases_lst = word_tokenize(str(target_noun_phrases[0]).strip())
                if len(target_noun_phrases_lst) > 1:
                    target_noun_phrases_lst_without_sw = [word for word in target_noun_phrases_lst if not word in all_stopwords]
                    target = ' '.join(target_noun_phrases_lst_without_sw)
                else:
                    target = target_noun_phrases_lst[0]

            if(len(source_noun_phrases) == 0):
                words = nltk.word_tokenize(after_like)
                tagged_words = pos_tagger.tag(words)
                for each in tagged_words:
                    if each[1] in nouns or each[1] in pronouns:
                        source = each[0]
            else:
                source_noun_phrases_lst = word_tokenize(str(source_noun_phrases[0]).strip())
                if len(source_noun_phrases_lst) > 1:
                    source_noun_phrases_lst_without_sw = [word for word in source_noun_phrases_lst if not word in all_stopwords]
                    source = ' '.join(source_noun_phrases_lst_without_sw)
                else:
                    source = source_noun_phrases_lst[0]

        print()
        print(target)
        print(attribute)
        print(source)

[('i', 'PRP')]



music


## Writing Target, Attribute and Source into excel file 

In [12]:
import xlwt
from xlwt import Workbook

In [13]:
# Workbook is created
wb2 = Workbook()

# add_sheet is used to create sheet.
sheet2 = wb2.add_sheet('Sheet 1', cell_overwrite_ok=True)

In [14]:
# Specifying style
style = xlwt.easyxf('font: bold 1')

In [15]:
sheet2.write(0, 0, 'Target', style)
sheet2.write(0, 1, 'Attribute', style)
sheet2.write(0, 2, 'Source', style)
sheet2.write(0, 3, 'labels', style)

In [ ]:
target = ""
attribute = ""
source = ""
row_index = 1

for i in range(df.shape[0]):
    # for metaphorical sentences
    if df.loc[i]['labels'] == 1:
        sentence = df.loc[i]['sentences'].lower()
        
        # for simile sentences having 'as' as a comparator
        if sentence.count(' as ') == 2:
            start = sentence.find(' as ')
            end = sentence.rfind(' as ')
            
            # +4 for skipping ' as '
            attribute = sentence[start+4:end].strip()
            
            target_noun_phrases = get_noun_phrase(sentence[:start])
            source_noun_phrases = get_noun_phrase(sentence[end+4:])

            if(len(target_noun_phrases) == 0):
                words = nltk.word_tokenize(sentence[:start])
                tagged_words = pos_tagger.tag(words)
                for each in tagged_words:
                    if each[1] in nouns or each[1] in pronouns:
                        target = each[0]
            else:
                target_noun_phrases_lst = word_tokenize(str(target_noun_phrases[0]).strip())
                if len(target_noun_phrases_lst) > 1:
                    target_noun_phrases_lst_without_sw = [word for word in target_noun_phrases_lst if not word in all_stopwords]
                    target = ' '.join(target_noun_phrases_lst_without_sw)
                else:
                    target = target_noun_phrases_lst[0]
            
            if(len(source_noun_phrases) == 0):
                words = nltk.word_tokenize(sentence[end+4:])
                tagged_words = pos_tagger.tag(words)
                for each in tagged_words:
                    if each[1] in nouns or each[1] in pronouns:
                        source = each[0]
            else:
                source_noun_phrases_lst = word_tokenize(str(source_noun_phrases[0]).strip())
                if len(source_noun_phrases_lst) > 1:
                    source_noun_phrases_lst_without_sw = [word for word in source_noun_phrases_lst if not word in all_stopwords]
                    source = ' '.join(source_noun_phrases_lst_without_sw)
                else:
                    source = source_noun_phrases_lst[0]
                
        # for simile sentences having 'like' as a comparator
        if sentence.count(' like ') == 1:
            like_index = sentence.find(' like ')
            before_like = sentence[:like_index]
            after_like = sentence[like_index+6:]

            words = nltk.word_tokenize(before_like)
            tagged_words = pos_tagger.tag(words)

            for each in tagged_words:
                if each[1] in adjectives:
                    attribute = each[0]
                elif each[1] in verbs:
                    present_form = WordNetLemmatizer().lemmatize(each[0],'v')
                    if present_form not in helping_verbs:
                        attribute = present_form

            attribute_index = before_like.find(attribute)
            target_noun_phrases = get_noun_phrase(before_like[:attribute_index])
            source_noun_phrases = get_noun_phrase(after_like)

            if(len(target_noun_phrases) == 0):
                words = nltk.word_tokenize(before_like[:attribute_index])
                tagged_words = pos_tagger.tag(words)
                for each in tagged_words:
                    if each[1] in nouns or each[1] in pronouns:
                        target = each[0]
            else:
                target_noun_phrases_lst = word_tokenize(str(target_noun_phrases[0]).strip())
                if len(target_noun_phrases_lst) > 1:
                    target_noun_phrases_lst_without_sw = [word for word in target_noun_phrases_lst if not word in all_stopwords]
                    target = ' '.join(target_noun_phrases_lst_without_sw)
                else:
                    target = target_noun_phrases_lst[0]

            if(len(source_noun_phrases) == 0):
                words = nltk.word_tokenize(after_like)
                tagged_words = pos_tagger.tag(words)
                for each in tagged_words:
                    if each[1] in nouns or each[1] in pronouns:
                        source = each[0]
            else:
                source_noun_phrases_lst = word_tokenize(str(source_noun_phrases[0]).strip())
                if len(source_noun_phrases_lst) > 1:
                    source_noun_phrases_lst_without_sw = [word for word in source_noun_phrases_lst if not word in all_stopwords]
                    source = ' '.join(source_noun_phrases_lst_without_sw)
                else:
                    source = source_noun_phrases_lst[0]

#         print()
#         print(target)
#         print(attribute)
#         print(source)
        sheet2.write(row_index, 0, target)
        sheet2.write(row_index, 1, attribute)
        sheet2.write(row_index, 2, source)
        sheet2.write(row_index, 3, 1)
        row_index += 1

In [28]:
wb2.save("knowledge_graph_triples.xls")

## Command for creating Knowledge Graph (Neo4j) 

In [12]:
# read by default 1st sheet of an excel file
d_frame = pandas.read_excel('knowledge_graph_triples.xls')

print(d_frame)
print(d_frame.shape)

         Target      Attribute    Source
0          face          black      coal
1         uncle          blind       bat
2         naina           cool  cucumber
3       soldier          brave      lion
4            he        cunning       fox
...         ...            ...       ...
1553       love  unconditional    warmth
1554    purring       soothing   lullaby
1555  small nut           hard      rock
1556  nut shell          tough   leather
1557        nut           rich    butter

[1558 rows x 3 columns]
(1558, 3)


In [13]:
# dropping duplicate rows from dataframe
d_frame = d_frame.drop_duplicates(ignore_index=True)

print(d_frame)
print(d_frame.shape)

         Target      Attribute    Source
0          face          black      coal
1         uncle          blind       bat
2         naina           cool  cucumber
3       soldier          brave      lion
4            he        cunning       fox
...         ...            ...       ...
1526       love  unconditional    warmth
1527    purring       soothing   lullaby
1528  small nut           hard      rock
1529  nut shell          tough   leather
1530        nut           rich    butter

[1531 rows x 3 columns]
(1531, 3)


In [37]:
with open("kb.txt", "w") as f:
    f.write('CREATE' + '\n')
    target_set = set()
    source_set = set()
    for i in range(d_frame.shape[0]):
        target_value = ""
        target_var = ""
        attribute_value = ""
        attribute_var = ""
        source_value = ""
        source_var = ""
        
        target_value = d_frame.loc[i][0]
        attribute_value = d_frame.loc[i][1]
        source_value = d_frame.loc[i][2]
        
        # '-' is to be replaced by ' ' as it is not allowed in variable name in Neo4j
        if target_value.find('-') != -1:
            target_value = target_value.replace('-', ' ')
        if attribute_value.find('-') != -1:
            attribute_value = attribute_value.replace('-', ' ')
        if source_value.find('-') != -1:
            source_value = source_value.replace('-', ' ')

        # "'" is to be replaced by "" as it is not allowed in variable name in Neo4j
        if target_value.find("'"):
            target_value = target_value.replace("'", "")
        if attribute_value.find("'"):
            attribute_value = attribute_value.replace("'", "")
        if attribute_value.find("'"):
            attribute_value = attribute_value.replace("'", "")

        target_var = '_'.join((target_value).split())
        attribute_var = '_'.join((attribute_value).split())
        source_var = '_'.join((source_value).split())
        
        target_var = target_var + "_t"
        if target_var not in target_set:
            f.write('(' + target_var + ':TARGET{value:"' + target_value + '"}),' + '\n')
            target_set.add(target_var)
        source_var = source_var + "_s"
        if source_var not in source_set:
            f.write('(' + source_var + ':SOURCE{value:"' + source_value + '"}),' + '\n')
            source_set.add(source_var)
        

        if i != d_frame.shape[0]-1:
            f.write('(' + source_var + ')-[:ATTRIBUTE {value:"' + attribute_value + '"}]->('+ target_var + '),' + '\n')
        else:
            f.write('(' + source_var + ')-[:ATTRIBUTE {value:"' + attribute_value + '"}]->('+ target_var + ');' + '\n')

In [14]:
java_path = "C:/Program Files/Java/jdk-18.0.1.1/bin/java.exe"
os.environ["JAVAHOME"] = java_path

In [15]:
jar = "C:/Users/Public/utility/stanford-postagger-full-2020-11-17/stanford-postagger.jar"
model = "C:/Users/Public/utility/stanford-postagger-full-2020-11-17/models/english-bidirectional-distsim.tagger"

In [16]:
pos_tagger = StanfordPOSTagger(model, jar, encoding = "utf-8")

In [17]:
with open("kb3.txt", "w") as f:
    f.write('CREATE' + '\n')
    target_set = set()
    source_set = set()
    for i in range(d_frame.shape[0]):
        target_value = ""
        target_var = ""
        attribute_value = ""
        attribute_var = ""
        source_value = ""
        source_var = ""
        
        target_value = d_frame.loc[i][0]
        attribute_value = d_frame.loc[i][1]
        source_value = d_frame.loc[i][2]
        
        # '-' is to be replaced by ' ' as it is not allowed in variable name in Neo4j
        if target_value.find('-') != -1:
            target_value = target_value.replace('-', ' ')
        if attribute_value.find('-') != -1:
            attribute_value = attribute_value.replace('-', ' ')
        if source_value.find('-') != -1:
            source_value = source_value.replace('-', ' ')

        # "'" is to be replaced by "" as it is not allowed in variable name in Neo4j
        if target_value.find("'"):
            target_value = target_value.replace("'", "")
        if attribute_value.find("'"):
            attribute_value = attribute_value.replace("'", "")
        if attribute_value.find("'"):
            attribute_value = attribute_value.replace("'", "")
            
        # Stanford POS tagging
        words = nltk.word_tokenize(target_value)
        target_tagged_words = pos_tagger.tag(words)
        
        words = nltk.word_tokenize(source_value)
        source_tagged_words = pos_tagger.tag(words)
        
        words = nltk.word_tokenize(attribute_value)
        attribute_tagged_words = pos_tagger.tag(words)

        target_var = '_'.join((target_value).split())
        attribute_var = '_'.join((attribute_value).split())
        source_var = '_'.join((source_value).split())
        
        target_var = target_var + "_t"
        if target_var not in target_set:
            f.write('(' + target_var + ':TARGET{value:"' + target_value + '",pos_tag:"' + target_tagged_words[0][-1] + '"}),' + '\n')
            target_set.add(target_var)
        source_var = source_var + "_s"
        if source_var not in source_set:
            f.write('(' + source_var + ':SOURCE{value:"' + source_value + '",pos_tag:"' + source_tagged_words[0][-1] + '"}),' + '\n')
            source_set.add(source_var)
        

        if i != d_frame.shape[0]-1:
            f.write('(' + source_var + ')-[:ATTRIBUTE {value:"' + attribute_value + '",pos_tag:"' + attribute_tagged_words[0][1] +  '"}]->('+ target_var + '),' + '\n')
        else:
            f.write('(' + source_var + ')-[:ATTRIBUTE {value:"' + attribute_value + '",pos_tag:"' + attribute_tagged_words[0][1] + '"}]->('+ target_var + ');' + '\n')